In [1]:
from selenium import webdriver
from pyvirtualdisplay import Display
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood
import dateutil.parser as dparser
import sunlight
from sunlight import congress
import re
from postal.parser import parse_address
import csv
import selenium
from bs4 import BeautifulSoup as soupy
import tweepy
import codecs
import io

In [2]:
action_categories = {'donation': ["donation","donate","contribute"],
                     'protest': ["protest","rally","demonstration"],
                     'indivisible': ["meeting","gathering","indivisible"],
                     'boycott': ["boycott"],
                     'advocate': ["contact your representative","contact your senator","petition",
                                 "call your representative","call your senator",
                                 "email your representative","email your senator"],
#                      'vote': ["vote","cast your ballot"]
#                      'town hall': ["town hall","open office","town meeting","townhall",'virtual townhall','listening session']
                          }

# action_categories = {'donation': ["donate"],
#                      'protest': ["protest"],
#                      'gathering': ["indivisible"],
#                      'boycott': ["boycott"],
#                      'advocate': ["representative","senator"],
#                      'vote': ["vote","cast your ballot"],
#                      'town hall': ["town hall",]
#                           }

issue_types = { 'immigrant rights' : ['immigrant rights',"immigrants' rights",'refugee rights', 'travel ban',
                                        'border wall','refugees','asylum','immigration reform',
                                        'immigrant advocacy','migrant rights','undocumented'],
               "women's rights" : ["women's rights","women's rights",'womens rights','gender equality',
                                  'girl power',"international women's day","war on women","planned parenthood"],
               "civil rights" : ["racial equality","black lives matter","african american rights","civil rights",
                                "black power","jim crow"],
               "LGTBQ rights" : ["marriage equality","transgender rights","equality act","lesbian rights",'gay rights',
                                'bisexual rights'],
               "voting rights": ["redistricting","gerrymandering","redistrict","gerrymander","voter id","voting access",
                                "voter access","voter suppression"],
               "worker rights": ['worker rights','right to work','minimum wage']
                      }

# issue_types = { 'immigrant rights' : ['immigrant rights'],
#                "women's rights" : ["women's rights"],
#                "civil rights" : ["civil rights"],
#                "LGTBQ rights" : ['gay rights','transgender rights'],
#                "voting rights": ["redistricting","gerrymandering","voter suppression"],
#                "worker rights": ['worker rights']
#                                       }

# organization_lists


In [3]:
config = configparser.ConfigParser()
config.read(os.path.join(os.pardir, 'config', 'ross.ini'))

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  from ipykernel import kernelapp as app


In [4]:
action_categories.keys()
print issue_types.keys()

['LGTBQ rights', 'worker rights', 'civil rights', 'immigrant rights', "women's rights", 'voting rights']


In [5]:
legislators = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'legislators.csv'))

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = legislators['title'] + ' ' + legislators['firstname'] + ' ' + legislators['lastname']

print legislators.columns
print
print list(legislators['title_name'])[:10]
legislator_list = list(legislators['title_name'])

Index([u'title', u'full_title', u'firstname', u'middlename', u'lastname',
       u'name_suffix', u'nickname', u'party', u'state', u'district',
       u'in_office', u'gender', u'phone', u'fax', u'website', u'webform',
       u'congress_office', u'bioguide_id', u'votesmart_id', u'fec_id',
       u'govtrack_id', u'crp_id', u'twitter_id', u'congresspedia_url',
       u'youtube_url', u'facebook_id', u'official_rss', u'senate_class',
       u'birthdate', u'oc_email', u'title_name'],
      dtype='object')

['Rep Robert Aderholt', 'Sen Lamar Alexander', 'Rep Justin Amash', 'Rep Mark Amodei', 'Rep Alma Adams', 'Rep Pete Aguilar', 'Rep Rick Allen', 'Rep Ralph Abraham', 'Rep Jodey Arrington', 'Rep Joe Barton']


In [6]:
immigration_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'immigration_orgs.csv'))
civil_rights_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'civil_rights_orgs.csv'))
lgbtq_orgs = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'lgbtq_orgs.csv'))

print lgbtq_orgs[:10]

# for org in immigration_orgs['org_name']:
#   print org


   Unnamed: 0                              org_name
0           0                       Accord Alliance
1           1                                ACT UP
2           2         Advocates for Informed Choice
3           3    Affirmation: Gay & Lesbian Mormons
4           4                      Against Equality
5           5  American Foundation for Equal Rights
6           6    American Veterans for Equal Rights
7           7                          Athlete Ally
8           8                        Atticus Circle
9           9                            Bash Back!


In [7]:
cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
cities['full_city'] = '\"' + cities['city'] + ', ' + cities['state'] + '\"'
print cities[:10]
city_list = list(cities['city'].str.rstrip())
state_list = list(cities['state'].str.rstrip())
# city_tuples = [(city, state) for city in city_list]
print city_list[:10]
# print city_list.index('Jersey City, NJ')

            city state population            full_city
0      New York     NY  8,363,710      "New York , NY"
1   Los Angeles     CA  3,833,995   "Los Angeles , CA"
2       Chicago     IL  2,853,114       "Chicago , IL"
3       Houston     TX  2,242,193       "Houston , TX"
4       Phoenix     AZ  1,567,924       "Phoenix , AZ"
5  Philadelphia     PA  1,447,395  "Philadelphia , PA"
6   San Antonio     TX  1,351,305   "San Antonio , TX"
7        Dallas     TX  1,279,910        "Dallas , TX"
8     San Diego     CA  1,279,329     "San Diego , CA"
9      San Jose     CA    948,279      "San Jose , CA"
['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San Antonio', 'Dallas', 'San Diego', 'San Jose']


In [8]:
keyword_searches = []
### Come up wiht long list of queries
for issue in issue_types.keys():
  for issue_keyword in issue_types[issue]:
    for combination in itertools.product([issue_keyword],city_list):
      keyword_searches.append(' '.join(list(combination)).decode('utf-8','ignore'))
      
for issue in issue_types.keys():
  for issue_keyword in issue_types[issue]:
    for category in action_categories.keys():
      for action_keyword in action_categories[category]:
        combination = [action_keyword,issue_keyword]
        keyword_searches.append(' '.join(combination).decode('utf-8','ignore'))

for org in immigration_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination).decode('utf-8','ignore'))

for org in civil_rights_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination).decode('utf-8','ignore'))
          
for org in lgbtq_orgs:
  for category in action_categories.keys():
    for action_keyword in action_categories[category]: 
      combination = [action_keyword,org]
      keyword_searches.append(' '.join(combination).decode('utf-8','ignore'))
          
for leg in legislator_list:
  for category in action_categories.keys():
    for category_keyword in action_categories[category]:
      combination = [leg,category_keyword]
      keyword_searches.append(' '.join(combination).decode('utf-8','ignore'))

print len(keyword_searches)
print keyword_searches[-20:]

220047
[u'Rep Lee Zeldin meeting', u'Rep Lee Zeldin gathering', u'Rep Lee Zeldin indivisible', u'Rep Ryan Zinke contact your representative', u'Rep Ryan Zinke contact your senator', u'Rep Ryan Zinke petition', u'Rep Ryan Zinke call your representative', u'Rep Ryan Zinke call your senator', u'Rep Ryan Zinke email your representative', u'Rep Ryan Zinke email your senator', u'Rep Ryan Zinke donation', u'Rep Ryan Zinke donate', u'Rep Ryan Zinke contribute', u'Rep Ryan Zinke boycott', u'Rep Ryan Zinke protest', u'Rep Ryan Zinke rally', u'Rep Ryan Zinke demonstration', u'Rep Ryan Zinke meeting', u'Rep Ryan Zinke gathering', u'Rep Ryan Zinke indivisible']


In [24]:
# today = datetime.datetime.today()
# with io.open('keyword_searches_shortest.txt', 'wb') as f:
#   for s in keyword_searches:
#     f.write(('"' + s.encode('ascii','ignore').rstrip() + '"' u','))
  

In [2]:
display = Display(visible=0, size=(800, 600))
display.start()
browser = webdriver.Firefox()

In [9]:
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=5)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth_handler=auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

def tweepy_this_id(tweet_id, action_keywords, issue_keywords, search_city):  
  tweet_status = api.get_status(int(tweet_id))
  tweet = tweet_status._json
  try:
    ### Extract URL from Tweet if it exists
    try:
      if tweet.get('entities',False).get('media',False).get('urls',False):
        tweet_urls = tweet['entities']['media']['urls']
      else:
        tweet_urls = ''
    except:
      tweet_urls = ''

  ### Extract Date from Tweet if it exists
    try:
      tweet_date = dparser.parse(tweet['text'],fuzzy=True).strftime("%m/%d/%Y")
    except:
      tweet_date = None
  ### Extract Address from Tweet if it exists
    try:
      address_components = {}
      for each in parse_address(tweet['text']):
        address_components[each[1]] = each[0]
      address = address_components.get('house_number','') + ' ' + address_components.get('road','')
      if search_city:
        city = search_city
      else:
        city = address_components.get('city',None)
      state = address_components.get('state',None)
      zipcode = address_components.get('postcode',None)
    except:
      address = ''
      city = None
      state = None
      zipcode = None
  ### Extract City and State from User Location
    try:
      if city == None:
        matches = re.findall("([\w\s]+),\s(\w+)", string)
        match = matches[0]
        tweet_city = match[0]
      else:
        tweet_city == city
      if state == None:
        matches = re.findall("([\w\s]+),\s(\w+)", string)
        match = matches[0]
        tweet_state = match[1]
      else:
        tweet_state = state
    except:
      tweet_city = ''
      tweet_state = ''
    ### Get Hashtags
    try:
      if tweet.get('entities',False).get('hashtags',False):
        tweet_hashtags = tweet['entities']['hashtags']
      else:
        tweet_hashtags = []
    except:
      tweet_hashtags = []
    ### Get User Mentions
    try:
      if tweet.get('entities',False).get('user_mentions',False):
        tweet_users = tweet['entities']['user_mentions']
      else:
        tweet_users = []
    except:
      tweet_users = []

    json_doc = {
                      'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                      'expiration_date': tweet_date,
                      'action_keywords': action_keywords, 
                      'issues_keywords': issue_keywords, 
                      'document': {'content': tweet['text'],
                                 'title': None,
                                 'summary': None,
                                 'url': 'https://twitter.com/statuses/' + str(tweet['id']),
                                 'date': tweet['created_at']
                              },
                      'location': {'addr1': address,
                                 'addr2': None,
                                 'city':  tweet_city,
                                 'state': tweet_state,
                                 'zip':  zipcode ,
                                 'lat':   None,
                                 'long':  None,
                                 },
                      'org_name': org,

                      # your mileage may vary
                      'other_metadata': {
                          'twitter_coordinates': tweet['coordinates'],
                          'twitter_urls': tweet_urls,
                          'twitter_user_mentions': tweet_users,
                          'twitter_hashtags': tweet_hashtags,
                          'twitter_favorite_count': tweet['favorite_count'],
                          'twitter_retweet_count': tweet['retweet_count']
                          }
                  }

    new_record = {
                          "_index" : "twitter",
                          "_type"  : "tweet",
                          "_id"    : tweet['id'],
                          "_source": json_doc,
                      }


    return new_record
  
  except Exception, e: # take care of all those ugly errors if there are some
    print(e)
    print tweet_id
    time.sleep(60)
    return None



In [ ]:
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=3)

# tweet_users = []
# tweet_timestamps = []
# tweet_actions = []
# tweet_issues= []

# for org in immigration_orgs['org_name']:

for city in city_list[5:]:
  with open(os.path.join(os.pardir, 'data/documents', 'tweepy_real_data_' + today.strftime("%Y%m%d%H") + '.txt'), 'a+') as outfile:
    for issue in issue_types:
      for issue_keyword in issue:
        for category in action_categories.keys():

            display = Display(visible=0, size=(800, 600))
            display.start()
            browser = webdriver.Firefox()

            tweet_ids = []
            tweet_texts = []

            action_list = action_categories[category]
            issue_list = issue_types[issue]

            tso = TwitterSearchOrder() # create a TwitterSearchOrder object
            tso.set_keywords(action_list, or_operator = True)
            tso.add_keyword('AND ' + issue_keyword)
            tso.add_keyword('AND ' + city)
            tso.set_language('en') # we want to see German tweets only
            tso.set_include_entities(True) # and don't give us all those entity information
            tso.set_result_type('recent')
            tso.set_count(30)
            querystr = tso.create_search_url() # create the url form the TwitterSearch package
            queryURL = 'https://twitter.com/search?l=&'+ querystr[1:] + '+since:'+ weeks_ago.strftime("%Y-%m-%d")
            # Use selenium to get the URL
            browser.get(queryURL)

#               # scroll to the bottom
            browser.execute_script("window.scrollTo(0, 1000000)")
            time.sleep(2)
    
            # grab the html
            html = browser.page_source
            # use bs4 on the html
            soup = soupy(html)

            tweets = soup.find_all('li','js-stream-item')
            for tweet in tweets:
              if tweet.find('p','tweet-text'):
      #           tweet_user = tweet.find('span','username').text
                tweet_text = tweet.find('p','tweet-text').text.encode('utf8')
                tweet_id = tweet['data-item-id']
      #           timestamp = tweet.find('a','tweet-timestamp')['title']
      #           tweet_timestamp = datetime.datetime.strptime(timestamp, '%I:%M %p - %d %b %Y')

                tweet_ids.append(tweet_id)
                tweet_texts.append(tweet_text)
      #           tweet_users.append(tweet_user)
      #           tweet_timestamps.append(tweet_timestamp)
      #           tweet_actions.append(action_list)
      #           tweet_issues.append(issue_list)



            tweets_df = pd.DataFrame(
                {'tweet_ids': tweet_ids,
                 'tweet_texts': tweet_texts
                })

            unique_tweets = tweets_df.drop_duplicates(subset=['tweet_texts'])

            tweet_id_list = list(set(list(unique_tweets['tweet_ids'])))

            if len(unique_tweets) > 0:
              try:
                for i,tweet_id in enumerate(tweet_id_list):
                  tweet_json = tweepy_this_id(tweet_id,action_list,issue_list, city)
                  if tweet_json:
                    json.dump(tweet_json, outfile)
                    outfile.write(',')
              except:
                print 'passed'
    #             print tweet_id
                pass
    #           if i % 5:
    #             print tweet_text
              time.sleep(5) # pause/sleeps for 5 seconds
              browser.quit()
              display.stop()
            else:
  #             print category + ' ' + city
  #             print 'no tweets to print'
  #             time.sleep(5) # pause/sleeps for 5 seconds
              browser.quit()
              display.stop()
              pass

#     outfile.seek(-1, os.SEEK_END)
#     outfile.truncate()
#     outfile.write(']')





In [ ]:
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=3)

# tweet_users = []
# tweet_timestamps = []
# tweet_actions = []
# tweet_issues= []

# for org in immigration_orgs['org_name']:

for keyword in keyword_searches:
  with open(os.path.join(os.pardir, 'data/documents', 'tweepy_real_data_' + today.strftime("%Y%m%d%H") + '.txt'), 'a+') as outfile:

    display = Display(visible=0, size=(800, 600))
    display.start()
    browser = webdriver.Firefox()

    tweet_ids = []
    tweet_texts = []

    action_list = action_categories[category]
    issue_list = issue_types[issue]

    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(keyword)
    tso.set_language('en') # we want to see German tweets only
    tso.set_include_entities(True) # and don't give us all those entity information
    tso.set_result_type('recent')
    tso.set_count(30)
    querystr = tso.create_search_url() # create the url form the TwitterSearch package
    queryURL = 'https://twitter.com/search?l=&'+ querystr[1:] + '+since:'+ weeks_ago.strftime("%Y-%m-%d")
    # Use selenium to get the URL
    browser.get(queryURL)
    for i in range (2):
      # scroll to the bottom
      browser.execute_script("window.scrollTo(0, 1000000)")
    # grab the html
    html = browser.page_source
    # use bs4 on the html
    soup = soupy(html)

    tweets = soup.find_all('li','js-stream-item')
    for tweet in tweets:
      if tweet.find('p','tweet-text'):
#           tweet_user = tweet.find('span','username').text
        tweet_text = tweet.find('p','tweet-text').text.encode('utf8')
        tweet_id = tweet['data-item-id']
#           timestamp = tweet.find('a','tweet-timestamp')['title']
#           tweet_timestamp = datetime.datetime.strptime(timestamp, '%I:%M %p - %d %b %Y')

        tweet_ids.append(tweet_id)
        tweet_texts.append(tweet_text)
#           tweet_users.append(tweet_user)
#           tweet_timestamps.append(tweet_timestamp)
#           tweet_actions.append(action_list)
#           tweet_issues.append(issue_list)



    tweets_df = pd.DataFrame(
        {'tweet_ids': tweet_ids,
         'tweet_texts': tweet_texts
        })

    unique_tweets = tweets_df.drop_duplicates(subset=['tweet_texts'])

    tweet_id_list = list(set(list(unique_tweets['tweet_ids'])))

    if len(unique_tweets) > 0:
      try:
        for i,tweet_id in enumerate(tweet_id_list):
          tweet_json = tweepy_this_id(tweet_id,keyword,keyword, None)
          if tweet_json:
            json.dump(tweet_json, outfile)
            outfile.write(',')
      except:
        print 'passed'
#             print tweet_id
        pass
#           if i % 5:
#             print tweet_text
      time.sleep(5) # pause/sleeps for 10 seconds
      browser.quit()
    else:
#             print category + ' ' + city
#             print 'no tweets to print'
#             time.sleep(5) # pause/sleeps for 10 seconds
      browser.quit()
      pass

#     outfile.seek(-1, os.SEEK_END)
#     outfile.truncate()
#     outfile.write(']')





In [5]:
today = datetime.datetime.today()
weeks_ago = today - datetime.timedelta(days=14)

tweet_ids = []
tweet_texts = []
tweet_users = []
tweet_timestamps = []
tweet_actions = []
tweet_issues= []

#### Immigration Orgs
for org in immigration_orgs['org_name']:
  for issue_keyword in issue_types['immigrant rights']:
    for category in action_categories.keys():
    
      action_list = action_categories[category]
      issue_list = issue_types['immigrant rights']

      tso = TwitterSearchOrder() # create a TwitterSearchOrder object
      tso.set_keywords(action_list, or_operator = True)
      tso.add_keyword('AND ' + issue_keyword)
      tso.add_keyword('AND ' + org)
      tso.set_language('en') 
      tso.set_include_entities(True) # and don't give us all those entity information
      tso.set_result_type('recent')
      tso.set_count(20)
      querystr = tso.create_search_url() # create the url form the TwitterSearch package
      queryURL = 'https://twitter.com/search?l=&'+ querystr[1:] + '+since:'+ weeks_ago.strftime("%Y-%m-%d")
      # Use selenium to get the URL
      browser.get(queryURL)
      for i in range (2):
        # scroll to the bottom
        browser.execute_script("window.scrollTo(0, 1000000)")
      # grab the html
      html = browser.page_source
      # use bs4 on the html
      soup = soupy(html)

      tweets = soup.find_all('li','js-stream-item')
      for tweet in tweets:
        if tweet.find('p','tweet-text'):
          tweet_user = tweet.find('span','username').text
          tweet_text = tweet.find('p','tweet-text').text.encode('utf8')
          tweet_id = tweet['data-item-id']
          timestamp = tweet.find('a','tweet-timestamp')['title']
          tweet_timestamp = datetime.datetime.strptime(timestamp, '%I:%M %p - %d %b %Y')

          tweet_ids.append(tweet_id)
          tweet_texts.append(tweet_text)
          tweet_users.append(tweet_user)
          tweet_timestamps.append(tweet_timestamp)
          tweet_actions.append(action_list)
          tweet_issues.append(issue_list)
        if len(tweet_ids) % 25:
          print 'In immigration orgs'
          print tweet_text

#### Civil Rights Orgs
for org in civil_rights_orgs['org_name']:
  for issue_keyword in issue_types['civil rights']:
    for category in action_categories.keys():
    
      action_list = action_categories[category]
      issue_list = issue_types['civil rights']

      tso = TwitterSearchOrder() # create a TwitterSearchOrder object
      tso.set_keywords(action_list, or_operator = True)
      tso.add_keyword('AND ' + issue_keyword)
      tso.add_keyword('AND ' + org)
      tso.set_language('en') 
      tso.set_include_entities(True) # and don't give us all those entity information
      tso.set_result_type('recent')
      tso.set_count(20)
      querystr = tso.create_search_url() # create the url form the TwitterSearch package
      queryURL = 'https://twitter.com/search?l=&'+ querystr[1:] + '+since:'+ weeks_ago.strftime("%Y-%m-%d")
      # Use selenium to get the URL
      browser.get(queryURL)
      for i in range (2):
        # scroll to the bottom
        browser.execute_script("window.scrollTo(0, 1000000)")
      # grab the html
      html = browser.page_source
      # use bs4 on the html
      soup = soupy(html)

      tweets = soup.find_all('li','js-stream-item')
      for tweet in tweets:
        if tweet.find('p','tweet-text'):
          tweet_user = tweet.find('span','username').text
          tweet_text = tweet.find('p','tweet-text').text.encode('utf8')
          tweet_id = tweet['data-item-id']
          timestamp = tweet.find('a','tweet-timestamp')['title']
          tweet_timestamp = datetime.datetime.strptime(timestamp, '%I:%M %p - %d %b %Y')

          tweet_ids.append(tweet_id)
          tweet_texts.append(tweet_text)
          tweet_users.append(tweet_user)
          tweet_timestamps.append(tweet_timestamp)
          tweet_actions.append(action_list)
          tweet_issues.append(issue_list)
        if len(tweet_ids) % 25:
          print 'In civil rights orgs'
          print tweet_text
          
#### LGTBQ  Orgs
for org in lgbtq_orgs['org_name']:
  for issue_keyword in issue_types['LGTBQ rights']:
    for category in action_categories.keys():
    
      action_list = action_categories[category]
      issue_list = issue_types['LGTBQ rights']

      tso = TwitterSearchOrder() # create a TwitterSearchOrder object
      tso.set_keywords(action_list, or_operator = True)
      tso.add_keyword('AND ' + issue_keyword)
      tso.add_keyword('AND ' + org)
      tso.set_language('en') 
      tso.set_include_entities(True) # and don't give us all those entity information
      tso.set_result_type('recent')
      tso.set_count(20)
      querystr = tso.create_search_url() # create the url form the TwitterSearch package
      queryURL = 'https://twitter.com/search?l=&'+ querystr[1:] + '+since:'+ weeks_ago.strftime("%Y-%m-%d")
      # Use selenium to get the URL
      browser.get(queryURL)
      for i in range (2):
        # scroll to the bottom
        browser.execute_script("window.scrollTo(0, 1000000)")
      # grab the html
      html = browser.page_source
      # use bs4 on the html
      soup = soupy(html)

      tweets = soup.find_all('li','js-stream-item')
      for tweet in tweets:
        if tweet.find('p','tweet-text'):
          tweet_user = tweet.find('span','username').text
          tweet_text = tweet.find('p','tweet-text').text.encode('utf8')
          tweet_id = tweet['data-item-id']
          timestamp = tweet.find('a','tweet-timestamp')['title']
          tweet_timestamp = datetime.datetime.strptime(timestamp, '%I:%M %p - %d %b %Y')

          tweet_ids.append(tweet_id)
          tweet_texts.append(tweet_text)
          tweet_users.append(tweet_user)
          tweet_timestamps.append(tweet_timestamp)
          tweet_actions.append(action_list)
          tweet_issues.append(issue_list)
        if len(tweet_ids) % 25:
          print 'In LGBTQ orgs'
          print tweet_text
          

tweets_df = pd.DataFrame(
    {'tweet_ids': tweet_ids,
     'tweet_texts': tweet_texts,
     'tweet_users': tweet_users,
     'tweet_timestamps': tweet_timestamps,
     'tweet_actions': tweet_actions,
     'tweet_issues': tweet_issues
    })

unique_tweets = tweets_df.drop_duplicates(subset=['tweet_ids'])

print unique_tweets[:10]





/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


KeyboardInterrupt: 

In [181]:
tweets_df = pd.DataFrame(
    {'tweet_ids': tweet_ids,
     'tweet_texts': tweet_texts,
     'tweet_users': tweet_users,
     'tweet_timestamps': tweet_timestamps,
     'tweet_actions': tweet_actions,
     'tweet_issues': tweet_issues
    })

unique_tweets = tweets_df.drop_duplicates(subset=['tweet_ids'])

print unique_tweets[:10]
print len(unique_tweets)


                                       tweet_actions           tweet_ids  \
0  [event, meetup, huddle, congregate, gather, ga...  215277827484958720   
1  [call, email, reach out, senator, representati...  780838022862471168   
2  [event, meetup, huddle, congregate, gather, ga...   14999715963740161   

                                        tweet_issues  \
0  [immigrant rights, immigrants' rights, refugee...   
1  [immigrant rights, immigrants' rights, refugee...   
2  [immigrant rights, immigrants' rights, refugee...   

                                         tweet_texts    tweet_timestamps  \
0  Tomorrow (20 June) #IOM and #UNHCR organize a ... 2012-06-19 20:00:00   
1  With H. E. Minister of Natural Disasters and R... 2016-09-27 11:34:00   
2  Representatives of EU agencies and #UNHCR gath... 2010-12-15 03:06:00   

       tweet_users  
0  @EvelienBorgman  
1          @nnnegm  
2      @RefugeesCE  
3


In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth_handler=auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

errors = 0

with open(os.path.join(os.pardir, 'data/documents', 'tweepy_real_data_' + today.strftime("%Y%m%d%H%M") + '.txt'), 'w') as outfile:
  outfile.write('[')
  for i,tweet_id in enumerate(list(unique_tweets.tweet_ids)):
    tweet_status = api.get_status(int(tweet_id))
    tweet = tweet_status._json
    ntweets = 0
    try:
      ### Extract URL from Tweet if it exists
      try:
        if tweet.get('entities',False).get('media',False).get('urls',False):
          tweet_urls = tweet['entities']['media']['urls']
        else:
          tweet_urls = ''
      except:
        tweet_urls = ''

    ### Extract Date from Tweet if it exists
      try:
        tweet_date = dparser.parse(tweet['text'],fuzzy=True).strftime("%m/%d/%Y")
      except:
        tweet_date = None
    ### Extract Address from Tweet if it exists
      try:
        address_components = {}
        for each in parse_address(tweet['text']):
          address_components[each[1]] = each[0]
        address = address_components.get('house_number','') + ' ' + address_components.get('road','')
        city = address_components.get('city',None)
        state = address_components.get('state',None)
        zipcode = address_components.get('postcode',None)
      except:
        address = ''
        city = None
        state = None
        zipcode = None
    ### Extract City and State from User Location
      try:
        if city == None:
          matches = re.findall("([\w\s]+),\s(\w+)", string)
          match = matches[0]
          tweet_city = match[0]
        if state == None:
          matches = re.findall("([\w\s]+),\s(\w+)", string)
          match = matches[0]
          tweet_state = match[1]
      except:
        tweet_city = ''
        tweet_state = ''
      ### Get Hashtags
      try:
        if tweet.get('entities',False).get('hashtags',False):
          tweet_hashtags = tweet['entities']['hashtags']
        else:
          tweet_hashtags = []
      except:
        tweet_hashtags = []
      ### Get User Mentions
      try:
        if tweet.get('entities',False).get('user_mentions',False):
          tweet_users = tweet['entities']['user_mentions']
        else:
          tweet_users = []
      except:
        tweet_users = []

      json_doc = {
                        'posted_date': datetime.datetime.today().strftime("%m/%d/%Y"),
                        'expiration_date': tweet_date,
                        'action_keywords': unique_tweets.iloc[[i],:].tweet_actions.values.tolist()[0], 
                        'issues_keywords': unique_tweets.iloc[[i],:].tweet_issues.values.tolist()[0], 
                        'document': {'content': tweet['text'],
                                   'title': None,
                                   'summary': None,
                                   'url': 'https://twitter.com/statuses/' + str(tweet['id']),
                                   'date': tweet['created_at']
                                },
                        'location': {'addr1': address,
                                   'addr2': None,
                                   'city':  tweet_city,
                                   'state': tweet_state,
                                   'zip':  zipcode ,
                                   'lat':   None,
                                   'long':  None,
                                   },
                        'org_name': org,

                        # your mileage may vary
                        'other_metadata': {
                            'twitter_coordinates': tweet['coordinates'],
                            'twitter_urls': tweet_urls,
                            'twitter_user_mentions': tweet_users,
                            'twitter_hashtags': tweet_hashtags,
                            'twitter_favorite_count': tweet['favorite_count'],
                            'twitter_retweet_count': tweet['retweet_count']
                            }
                    }

      new_record = {
                            "_index" : "twitter",
                            "_type"  : "tweet",
                            "_id"    : tweet['id'],
                            "_source": json_doc,
                        }

      json.dump(new_record, outfile)
      outfile.write(',')
      ntweets += 1
      if ntweets % 100:
        print( tweet['text']  )

    except error as e: # take care of all those ugly errors if there are some
      print(e)
      errors += 1
      if errors == 5:
        break
      time.sleep(60*3)

  outfile.seek(-1, os.SEEK_END)
  outfile.truncate()
  outfile.write(']')

